# Regime-Conditioned Gating (BTCUSDT 1h)

**Idea:** the optimal similarity threshold *and even the best simple policy* can depend on the market regime.

**Artifacts:**
- table of optimal `(regime → threshold, policy)`
- comparison: **regime-aware** vs **single global** gating

Notes:
- If the API returns `episode.regime`, we use it. Otherwise we compute a simple proxy regime from episode returns (trend + volatility).
- Evaluation here is on the **retrieved episode distribution** at many anchors (not a true historical market replay).


In [14]:
import os
import time
from datetime import datetime, timezone

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from aipricepatterns import Client

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)

def safe_float(x, default=np.nan) -> float:
    try:
        return float(x)
    except Exception:
        return float(default)

def map_suggested_action_to_pos(x) -> int:
    if x is None:
        return 0
    if isinstance(x, (int, float)):
        v = int(x)
        if v in (-1, 0, 1):
            return v
        # sometimes encoded as 0/1/2
        if v in (0, 1, 2):
            return 1 if v == 1 else (-1 if v == 2 else 0)
        return 0
    s = str(x).strip().lower()
    if s in ('hold','flat','none','neutral','wait'): return 0
    if s in ('long','buy','bull','up'): return 1
    if s in ('short','sell','bear','down'): return -1
    return 0

def episode_horizon_pnl(ep: dict, horizon: int, pos: int, trade_cost_pct: float) -> float:
    ts = ep.get("transitions")
    if not isinstance(ts, list) or not ts:
        return 0.0
    steps = min(int(horizon), len(ts))
    pnl = 0.0
    for i in range(steps):
        t = ts[i] if isinstance(ts[i], dict) else {}
        pnl += float(pos) * safe_float(t.get("ret", t.get("return", 0.0)), 0.0)
    # simple entry cost once (0 -> pos)
    pnl -= abs(int(pos)) * (trade_cost_pct / 100.0)
    return float(pnl)

def episode_policy_pnl(ep: dict, horizon: int, trade_cost_pct: float, policy: str) -> float:
    ts = ep.get("transitions")
    if not isinstance(ts, list) or not ts:
        return 0.0
    if policy == "flat":
        return 0.0
    if policy == "long":
        return episode_horizon_pnl(ep, horizon=horizon, pos=1, trade_cost_pct=trade_cost_pct)
    if policy == "short":
        return episode_horizon_pnl(ep, horizon=horizon, pos=-1, trade_cost_pct=trade_cost_pct)
    if policy == "suggested":
        t0 = ts[0] if isinstance(ts[0], dict) else {}
        pos = map_suggested_action_to_pos(t0.get("suggestedAction"))
        return episode_horizon_pnl(ep, horizon=horizon, pos=pos, trade_cost_pct=trade_cost_pct)
    return 0.0

def detect_regime(ep: dict, vol_thr: float = 0.02, drift_thr: float = 0.01) -> str:
    r = ep.get("regime")
    if r is not None and str(r).strip() != "":
        return str(r)
    ts = ep.get("transitions")
    if not isinstance(ts, list) or not ts:
        return "Unknown"
    rets = [safe_float(t.get("ret", 0.0), 0.0) for t in ts if isinstance(t, dict)]
    if not rets:
        return "Unknown"
    cum_ret = float(np.sum(rets))
    vol = float(np.std(rets)) if len(rets) > 1 else 0.0
    if vol >= vol_thr:
        return "HighVol"
    if cum_ret >= drift_thr:
        return "Bull"
    if cum_ret <= -drift_thr:
        return "Bear"
    return "Sideways"


## Parameters
Defaults: 300 anchors, BTCUSDT 1h. Reduce `EPISODES_PER_ANCHOR` for speed.


In [15]:
from pathlib import Path

BASE_URL = os.getenv("AIPP_BASE_URL", "https://aipricepatterns.com/api/rust")
API_KEY = os.getenv("AIPP_API_KEY")

SYMBOL = os.getenv("AIPP_RL_SYMBOL", "BTCUSDT")
INTERVAL = os.getenv("AIPP_RL_INTERVAL", "1h")

ANCHOR_POINTS = int(os.getenv("AIPP_SWEEP_ANCHORS", "300"))
LOOKBACK_DAYS = int(os.getenv("AIPP_SWEEP_LOOKBACK_DAYS", "120"))

FORECAST_HORIZON = int(os.getenv("AIPP_RL_HORIZON", "24"))
EPISODES_PER_ANCHOR = int(os.getenv("AIPP_SWEEP_EPISODES_PER_ANCHOR", "60"))
MIN_SIMILARITY = float(os.getenv("AIPP_RL_MIN_SIMILARITY", "0.70"))
SAMPLING_STRATEGY = os.getenv("AIPP_RL_SAMPLING_STRATEGY", "uniform")

TRADE_COST_PCT = float(os.getenv("AIPP_RL_TRADE_COST_PCT", "0.00"))

# Regime heuristic thresholds (only used if episode.regime is missing)
REGIME_VOL_THR = float(os.getenv("AIPP_REGIME_VOL_THR", "0.02"))
REGIME_DRIFT_THR = float(os.getenv("AIPP_REGIME_DRIFT_THR", "0.01"))

# Optimization constraints
MIN_COVERAGE = float(os.getenv("AIPP_REGIME_MIN_COVERAGE", "0.05"))
MIN_TRADES = int(os.getenv("AIPP_REGIME_MIN_TRADES", "200"))

# Cache location: by default keep _cache next to this notebook folder
NOTEBOOK_DIR = Path.cwd()
DEFAULT_CACHE_DIR = NOTEBOOK_DIR / "_cache"
CACHE_DIR = Path(os.getenv("AIPP_RESEARCH_CACHE_DIR", str(DEFAULT_CACHE_DIR)))
CACHE_PATH = CACHE_DIR / f"06_regime_eps_{SYMBOL}_{INTERVAL}_{ANCHOR_POINTS}.csv"

# Backward-compat: if you previously ran notebooks from inside python-sdk/research,
# you may have a nested python-sdk/research/_cache folder.
ALT_CACHE_PATH = NOTEBOOK_DIR / "python-sdk" / "research" / "_cache" / CACHE_PATH.name
if not CACHE_PATH.exists() and ALT_CACHE_PATH.exists():
    print("Using legacy nested cache path:", ALT_CACHE_PATH)
    CACHE_PATH = ALT_CACHE_PATH
    CACHE_DIR = ALT_CACHE_PATH.parent

print("Base URL:", BASE_URL)
print(f"Symbol: {SYMBOL}  Interval: {INTERVAL}")
print(f"Anchors: {ANCHOR_POINTS}  LookbackDays: {LOOKBACK_DAYS}")
print(f"Episodes/anchor: {EPISODES_PER_ANCHOR}  minSimilarity: {MIN_SIMILARITY:.2f}  horizon: {FORECAST_HORIZON}")
print(f"TradeCostPct: {TRADE_COST_PCT:.4f}%")
print(f"Constraints: MIN_COVERAGE={MIN_COVERAGE:.2%}  MIN_TRADES={MIN_TRADES}")
print("Cache:", str(CACHE_PATH))


Using legacy nested cache path: /Users/serg/projects/prod/ai_patterns/python-sdk/research/python-sdk/research/_cache/06_regime_eps_BTCUSDT_1h_300.csv
Base URL: https://aipricepatterns.com/api/rust
Symbol: BTCUSDT  Interval: 1h
Anchors: 300  LookbackDays: 120
Episodes/anchor: 60  minSimilarity: 0.70  horizon: 24
TradeCostPct: 0.0000%
Constraints: MIN_COVERAGE=5.00%  MIN_TRADES=200
Cache: /Users/serg/projects/prod/ai_patterns/python-sdk/research/python-sdk/research/_cache/06_regime_eps_BTCUSDT_1h_300.csv


## Data collection (cached)
We fetch episodes across anchors and compute regime + PnL for multiple simple policies.


In [16]:
CACHE_DIR.mkdir(parents=True, exist_ok=True)

if CACHE_PATH.exists():
    df = pd.read_csv(CACHE_PATH)
    print("loaded cache:", str(CACHE_PATH), "rows:", len(df))
else:
    client = Client(base_url=BASE_URL, api_key=API_KEY)
    now_ms = int(time.time() * 1000)
    start_ms = now_ms - LOOKBACK_DAYS * 24 * 60 * 60 * 1000
    anchors = np.linspace(start_ms, now_ms, num=ANCHOR_POINTS, dtype=np.int64).tolist()
    rows = []
    for idx, anchor_ts in enumerate(anchors, start=1):
        res = client.get_rl_episodes(
            symbol=SYMBOL,
            interval=INTERVAL,
            anchor_ts=int(anchor_ts),
            forecast_horizon=FORECAST_HORIZON,
            num_episodes=EPISODES_PER_ANCHOR,
            min_similarity=MIN_SIMILARITY,
            include_actions=True,
            reward_type="returns",
            sampling_strategy=SAMPLING_STRATEGY,
        )
        eps = res.get("episodes") if isinstance(res, dict) else None
        if not isinstance(eps, list) or not eps:
            continue
        anchor_dt = datetime.fromtimestamp(anchor_ts/1000, tz=timezone.utc).strftime("%Y-%m-%d %H:%M")
        for ep in eps:
            sim = safe_float(ep.get("similarity"), np.nan)
            regime = detect_regime(ep, vol_thr=REGIME_VOL_THR, drift_thr=REGIME_DRIFT_THR)
            rows.append({
                "anchorTs": int(anchor_ts),
                "anchorDtUtc": anchor_dt,
                "similarity": float(sim),
                "regime": str(regime),
                "pnl_suggested": episode_policy_pnl(ep, FORECAST_HORIZON, TRADE_COST_PCT, "suggested"),
                "pnl_long": episode_policy_pnl(ep, FORECAST_HORIZON, TRADE_COST_PCT, "long"),
                "pnl_short": episode_policy_pnl(ep, FORECAST_HORIZON, TRADE_COST_PCT, "short"),
            })
        if idx % 20 == 0:
            print(f"{idx}/{len(anchors)} anchors... rows={len(rows)}")
    df = pd.DataFrame(rows)
    df = df.dropna(subset=["similarity"]).reset_index(drop=True)
    df.to_csv(CACHE_PATH, index=False)
    print("wrote cache:", str(CACHE_PATH), "rows:", len(df))

# Normalize older cache schemas
if "pnl_suggested" not in df.columns and "pnl" in df.columns:
    df = df.rename(columns={"pnl": "pnl_suggested"})
if "regime" not in df.columns:
    df["regime"] = "Unknown"
if "pnl_long" not in df.columns:
    df["pnl_long"] = np.nan
if "pnl_short" not in df.columns:
    df["pnl_short"] = np.nan
df["pnl_flat"] = 0.0

print("columns:", list(df.columns))
df.head()


loaded cache: /Users/serg/projects/prod/ai_patterns/python-sdk/research/python-sdk/research/_cache/06_regime_eps_BTCUSDT_1h_300.csv rows: 17957
columns: ['anchorTs', 'similarity', 'pnl_suggested', 'regime', 'pnl_long', 'pnl_short', 'pnl_flat']


,anchorTs,similarity,pnl_suggested,regime,pnl_long,pnl_short,pnl_flat
0,1755797858885,0.9248,15.6328,HighVol,NaN,NaN,0.0
1,1755797858885,0.9212,0.0000,HighVol,NaN,NaN,0.0
2,1755797858885,0.9186,0.0000,HighVol,NaN,NaN,0.0
3,1755797858885,0.9148,0.0000,HighVol,NaN,NaN,0.0
4,1755797858885,0.9147,0.0000,HighVol,NaN,NaN,0.0


## Regime overview
If the regime labels look wrong, either the API did not provide `episode.regime` or the heuristic thresholds need tuning.


In [17]:
pnl_col = "pnl_suggested" if "pnl_suggested" in df.columns else ("pnl" if "pnl" in df.columns else None)
if pnl_col is None:
    raise ValueError(f"No PnL column found. Available columns: {list(df.columns)}")

display_df = (
    df.groupby("regime")
      .agg(
          n=("similarity","size"),
          sim_mean=("similarity","mean"),
          pnl_mean=(pnl_col,"mean"),
      )
      .sort_values("n", ascending=False)
      .reset_index()
)
display_df


,regime,n,sim_mean,pnl_mean
0,HighVol,17957,0.933395,0.31732


In [18]:
fig = px.box(df, x="regime", y="similarity", title="Similarity distribution by regime")
fig.update_layout(height=380)
fig


In [19]:
pnl_col = "pnl_suggested" if "pnl_suggested" in df.columns else ("pnl" if "pnl" in df.columns else None)
fig = px.box(df, x="regime", y=pnl_col, title=f"PnL distribution by regime ({pnl_col})")
fig.update_layout(height=380)
fig


## Optimize threshold + policy per regime
We choose `(threshold, policy)` to maximize **overall expected PnL** (treating non-trades as 0 PnL), subject to:
- minimum coverage per regime: `MIN_COVERAGE`
- minimum trades per regime: `MIN_TRADES`


In [20]:
policies = []
if "pnl_suggested" in df.columns: policies.append("suggested")
if "pnl_long" in df.columns and df["pnl_long"].notna().any(): policies.append("long")
if "pnl_short" in df.columns and df["pnl_short"].notna().any(): policies.append("short")
policies.append("flat")

if df["regime"].nunique() <= 1:
    print("WARNING: Only 1 regime detected:", df["regime"].unique())
    print("This means regime-conditioning cannot be demonstrated yet; tune REGIME_VOL_THR/REGIME_DRIFT_THR or use API-provided episode.regime.")

thr_grid = np.round(np.linspace(max(0.5, df["similarity"].quantile(0.10)), min(0.995, df["similarity"].quantile(0.99)), 30), 4)

rows = []
for regime in sorted(df["regime"].unique()):
    rdf = df[df["regime"] == regime].copy()
    if len(rdf) < max(MIN_TRADES, 50):
        continue
    for t in thr_grid:
        m = rdf["similarity"] >= float(t)
        trades = int(m.sum())
        coverage = float(m.mean())
        if trades < MIN_TRADES or coverage < MIN_COVERAGE:
            continue
        for pol in policies:
            pnl_col = f"pnl_{pol}" if pol != "flat" else "pnl_flat"
            if pnl_col not in rdf.columns:
                continue
            # overall expected pnl: trade pnl when m else 0
            overall = float(rdf.loc[m, pnl_col].sum() / len(rdf))
            trade_avg = float(rdf.loc[m, pnl_col].mean())
            winrate = float((rdf.loc[m, pnl_col] > 0).mean())
            rows.append({
                "regime": regime,
                "threshold": float(t),
                "policy": pol,
                "overallAvgPnL": overall,
                "tradeAvgPnL": trade_avg,
                "winrate": winrate,
                "trades": trades,
                "coverage": coverage,
                "n": int(len(rdf)),
            })

grid = pd.DataFrame(rows)
if len(grid) == 0:
    raise ValueError("No configs satisfy constraints. Try lowering MIN_TRADES/MIN_COVERAGE.")
best_regime = (
    grid.sort_values(["regime","overallAvgPnL","coverage"], ascending=[True, False, False])
        .groupby("regime").head(1).reset_index(drop=True)
)
best_regime


This means regime-conditioning cannot be demonstrated yet; tune REGIME_VOL_THR/REGIME_DRIFT_THR or use API-provided episode.regime.


,regime,threshold,policy,overallAvgPnL,tradeAvgPnL,winrate,trades,coverage,n
0,HighVol,0.8693,suggested,0.290299,0.32256,0.185446,16161,0.899983,17957


In [21]:
fig = px.bar(best_regime, x="regime", y="threshold", color="policy", title="Optimal threshold by regime (colored by chosen policy)")
fig.update_layout(height=420)
fig


## Compare: regime-aware vs single global gating
We also optimize a **single global** `(threshold, policy)` on the pooled dataset under the same constraints.


In [22]:
def pick_best_global(df_all: pd.DataFrame, thr_grid: np.ndarray) -> pd.Series:
    rows = []
    for t in thr_grid:
        m = df_all["similarity"] >= float(t)
        trades = int(m.sum())
        coverage = float(m.mean())
        if trades < MIN_TRADES or coverage < MIN_COVERAGE:
            continue
        for pol in ["suggested","long","short","flat"]:
            pnl_col = f"pnl_{pol}" if pol != "flat" else "pnl_flat"
            overall = float(df_all.loc[m, pnl_col].sum() / len(df_all))
            trade_avg = float(df_all.loc[m, pnl_col].mean())
            winrate = float((df_all.loc[m, pnl_col] > 0).mean())
            rows.append({
                "threshold": float(t),
                "policy": pol,
                "overallAvgPnL": overall,
                "tradeAvgPnL": trade_avg,
                "winrate": winrate,
                "trades": trades,
                "coverage": coverage,
            })
    g = pd.DataFrame(rows)
    if len(g) == 0:
        raise ValueError("No global configs satisfy constraints; loosen MIN_TRADES/MIN_COVERAGE.")
    return g.sort_values(["overallAvgPnL","coverage"], ascending=[False, False]).iloc[0]

best_global = pick_best_global(df, thr_grid)
best_global


threshold           0.8693
policy           suggested
overallAvgPnL     0.290299
tradeAvgPnL        0.32256
winrate           0.185446
trades               16161
coverage          0.899983
Name: 0, dtype: object

In [23]:
def score_regime_aware(df_all: pd.DataFrame, best_regime: pd.DataFrame) -> dict:
    mask = np.zeros(len(df_all), dtype=bool)
    pnl = np.zeros(len(df_all), dtype=float)
    for _, r in best_regime.iterrows():
        regime = r["regime"]
        t = float(r["threshold"])
        pol = str(r["policy"])
        pnl_col = f"pnl_{pol}" if pol != "flat" else "pnl_flat"
        m = (df_all["regime"] == regime) & (df_all["similarity"] >= t)
        mask |= m.to_numpy()
        pnl[m.to_numpy()] = df_all.loc[m, pnl_col].to_numpy(dtype=float)
    return {
        "coverage": float(mask.mean()),
        "trades": int(mask.sum()),
        "overallAvgPnL": float(pnl.sum() / len(df_all)),
        "tradeAvgPnL": float(pnl[mask].mean()) if mask.any() else float("nan"),
        "winrate": float((pnl[mask] > 0).mean()) if mask.any() else float("nan"),
    }

def score_global(df_all: pd.DataFrame, best_global: pd.Series) -> dict:
    t = float(best_global["threshold"])
    pol = str(best_global["policy"])
    pnl_col = f"pnl_{pol}" if pol != "flat" else "pnl_flat"
    m = df_all["similarity"] >= t
    pnl = np.zeros(len(df_all), dtype=float)
    pnl[m.to_numpy()] = df_all.loc[m, pnl_col].to_numpy(dtype=float)
    return {
        "coverage": float(m.mean()),
        "trades": int(m.sum()),
        "overallAvgPnL": float(pnl.sum() / len(df_all)),
        "tradeAvgPnL": float(pnl[m.to_numpy()].mean()) if m.any() else float("nan"),
        "winrate": float((pnl[m.to_numpy()] > 0).mean()) if m.any() else float("nan"),
    }

aware = score_regime_aware(df, best_regime)
global_ = score_global(df, best_global)

summary = pd.DataFrame([
    {"policy": "single-global", **global_},
    {"policy": "regime-aware", **aware},
])
summary


,policy,coverage,trades,overallAvgPnL,tradeAvgPnL,winrate
0,single-global,0.899983,16161,0.290299,0.32256,0.185446
1,regime-aware,0.899983,16161,0.290299,0.32256,0.185446


In [24]:
fig = px.bar(summary, x="policy", y="overallAvgPnL", color="policy", title="Overall expected PnL: regime-aware vs single-global")
fig.update_layout(height=360)
fig


## How to interpret the results (what I will check in your outputs)
- If some regimes have very few trades, their “optimal threshold” is likely noise → increase `MIN_TRADES` or merge regimes.
- Look at **overallAvgPnL** (includes 0 for non-trades) for a fair coverage/quality tradeoff.
- If regime-aware improves overallAvgPnL without collapsing coverage, that’s a strong product story: “adaptive confidence gating”.
